# Copy SecurityDatabase.mdb files to local PCs
This script runs the following processes: Copies CPC's SecurityDatabase.mdb from Z:\User to the local machines

## Setup

### Import Libraries

In [25]:
print("Initializing Python script...\n")
print("Importing libraries...\n")
# Standard Library Imports
import os
import shutil
import sys
from datetime import datetime
import logging
from logging.handlers import RotatingFileHandler
from pathlib import Path

# Third-Party Imports
import pandas as pd
from sqlalchemy import inspect

# Project (Local and Global) Imports
base = Path().resolve().parents[1] # monorepo root
sys.path.insert(0, str(base))
print(base)

from shared_config.src.shared_config.sql import PROD_SERVER as server, PYRO_SERVER as pyro_server, PYRO_DATABASE as database
from shared_utils.src.shared_utils.sql import get_engine, read_table, write_to_sql

Initializing Python script...

Importing libraries...

C:\python


### Settings

#### Define Notebook Settings

In [26]:
# Toggle max rows and max columns
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Setup Logging

In [27]:
base = Path().resolve().parents[1]

log_dir = base / 'logs'
log_dir.mkdir(parents=True, exist_ok=True)

log_file = log_dir / f'update_security_databases_{datetime.now():%Y%m%d}.log'

logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s = %(message)s',
    handlers=[
        RotatingFileHandler(log_file, maxBytes=10*1024*1024, backupCount=5),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

logger.info('=' * 60)
logger.info('START CURE PC AUDIT SCRIPT')
logger.info('=' * 60)
logger.info(f'Start Time: {datetime.now():%Y-%m-%d %H:%M:%S}')
logger.info(f'Log File: {log_file}')

print(f'\nLogging to: {log_file}\n')

INFO = ============================================================
INFO = START CURE PC AUDIT SCRIPT
INFO = ============================================================
INFO = Start Time: 2026-01-29 11:08:38
INFO = Log File: C:\python\logs\update_security_databases_20260129.log



Logging to: C:\python\logs\update_security_databases_20260129.log



#### Set CPC Variables

In [28]:
cpc_client = r'C:\Program Files\CPC Client'
cpc_server = r'C:\Program Files\CPC ObjServer'

file = 'SecurityDatabase.mdb'

source = r'\\ut02sa7\ac\User'

### Connect to Pyrometry Database on Production SQL Server

In [29]:
print(f'Connecting to {server} server {database} database...\n')

engine = get_engine(server, database)

equip_pcs = read_table(engine, 'SELECT * FROM [Equipment PCs]')

inspector = inspect(engine)
inspector.get_table_names()

equip_pcs

Connecting to UT02SQLPRODAGL server Pyrometry database...



,PC,PCID,IP_Address,Site,Equip,Alt_Name,Equip_Name,DAT_Name,Equip_Type,CPC_Version,Database_Name,SL_Number
0,3x4-A,ITE04024979,10.117.5.214,Dayton,None,3x4,None,None,Autoclave,2023-03-13,CPC_3x4,NA
1,3x4-B,ITE04020005,10.117.5.221,Dayton,None,3x4,None,None,Autoclave,2023-03-13,CPC_3x4,NA
2,4x9-A,ITE04020008,10.117.5.217,Dayton,None,4x9,None,None,Autoclave,2023-03-13,CPC_4x9,NA
3,4x9-B,ITE04024982,10.117.5.218,Dayton,None,4x9,None,None,Autoclave,2023-03-13,CPC_4x9,NA
4,10x30-A,ITE04023021,10.117.5.246,Dayton,None,10x30,None,None,Autoclave,2023-03-13,CPC_10x30,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
73,OV59-B,ITE00630608,10.82.27.149,G12,Oven 59,OV59,Oven59,OV59,Oven,2023-03-13,CPC_OV59,SL244858
74,Old Lewco-A,ITE04020032,10.117.160.62,Dayton,None,Old Lewco,None,None,Oven,2023-03-13,CPC_OldLewco,NA
75,Old Lewco-B,ITE04020009,10.117.161.2,Dayton,None,Old Lewco,None,None,Oven,2023-03-13,CPC_OldLewco,NA
76,Spike Oven-A,ITE04020543,10.82.243.136,ACCE,Spike Oven,Spike Oven,SpikeOven,SpikeOven,Oven,2023-03-13,CPC_SpikeOven,SL237055


## Functions

### Check for SecurityDatabase.mdb

In [30]:
def find_local_security_db(pc, ip, pcid, file):
    """Try to find the SecurityDatabase.mdb file for the given PC."""
    
    destinations = [
        r'C$\Program Files\CPC Client',
        r'C$\Program Files\CPC ObjServer',
    ]
    
    for identifier in [ip, pcid]:
        for dest in destinations:
            path = rf'{identifier}\{dest}\{file}'
            if os.path.exists(path):
                print(f'Found {file} for {pc} at {path}')
                return path
            
    print(f'Could not find {file} for {pc}')
    return None

In [31]:
def find_new_security_db(equip, site, database_source, file):
    """Find SecurityDatabase.mdb to be copied to local PC"""
    
    if site == 'Dayton':
        return None, None
    
    site_map = {
        'ACCE': 'ACCE',
        'East Gate': 'EastGate',
        'H5': 'IRAD',
        'HTS': 'HTS',
    }
    
    if site in site_map:
        database = site_map[site]
        source = rf'{database_source}\{database}\{file}'
    elif site in ('G12', 'G14'):
        database = 'Oven22' if equip == 'OV52' else 'ACS'
    elif site == 'FCC':
        database = 'Oven22' if equip == 'OV22' else 'FCC'
    else:
        print(f'Could not find security database for {equip} at {site}')
        return None, None
    
    source = rf'{database_source}\{database}\{file}'
    return database, source

## Run Code

### Extract Info from Local PCs

In [ ]:
logger.info('Starting PC information extraction...')

try:
    total = len(equip_pcs)
    success_count = 0
    failure_count = 0
    
    # Find SecurityDatabase.mdb
    for i, (index, row) in enumerate(equip_pcs.iterrows(), 1):
        pc = row['PC']
        ip = row['IP_Address']
        pcid = row['PCID']
        equip = row['Alt_Name']
        site = row['Site']
        
        print(f'Processing {i}/{total}: {pc}')
        
        security_db = find_local_security_db(pc, ip, pcid, file)
        database, db_source = find_new_security_db(equip, site, source, file)
        
        if database and db_source:
            try:
                destination_path = rf'\\ut02sa7\ac\~Cody\Testing\{database}\{pc}'
                os.makedirs(destination_path, exist_ok=True)
                # shutil.copy2(db_source, destination_path)
                success_count += 1
                logger.info(f'Successfully copied {file} for {pc} to {destination_path}')
            except Exception as e:
                failure_count += 1
                logger.error(f'Failed to copy {file} for {pc}: {e}')
    logger.info('=' * 60)
    logger.info(f' Finished copying {file} to local machines. Success: {success_count}, Failure: {failure_count}')
    logger.info('=' * 60)
    
except Exception as e:
    logger.error(f'Error during PC information extraction: {e}')
    raise

INFO = Starting PC information extraction...
INFO = Successfully copies SecurityDatabase.mdb for AC1-A to \\ut02sa7\ac\~Cody\Testing\FCC\AC1-A


Processing 1/78: 3x4-A
Could not find SecurityDatabase.mdb for 3x4-A
Processing 2/78: 3x4-B
Could not find SecurityDatabase.mdb for 3x4-B
Processing 3/78: 4x9-A
Could not find SecurityDatabase.mdb for 4x9-A
Processing 4/78: 4x9-B
Could not find SecurityDatabase.mdb for 4x9-B
Processing 5/78: 10x30-A
Could not find SecurityDatabase.mdb for 10x30-A
Processing 6/78: 10x30-B
Could not find SecurityDatabase.mdb for 10x30-B
Processing 7/78: 18x35-A
Could not find SecurityDatabase.mdb for 18x35-A
Processing 8/78: 18x35-B
Could not find SecurityDatabase.mdb for 18x35-B
Processing 9/78: AC1-A
Could not find SecurityDatabase.mdb for AC1-A


INFO = Successfully copies SecurityDatabase.mdb for AC1-B to \\ut02sa7\ac\~Cody\Testing\FCC\AC1-B
INFO = Successfully copies SecurityDatabase.mdb for AC2-A to \\ut02sa7\ac\~Cody\Testing\FCC\AC2-A
INFO = Successfully copies SecurityDatabase.mdb for AC2-B to \\ut02sa7\ac\~Cody\Testing\FCC\AC2-B
INFO = Successfully copies SecurityDatabase.mdb for AC3-A to \\ut02sa7\ac\~Cody\Testing\FCC\AC3-A
INFO = Successfully copies SecurityDatabase.mdb for AC3-B to \\ut02sa7\ac\~Cody\Testing\FCC\AC3-B


Processing 10/78: AC1-B
Could not find SecurityDatabase.mdb for AC1-B
Processing 11/78: AC2-A
Could not find SecurityDatabase.mdb for AC2-A
Processing 12/78: AC2-B
Could not find SecurityDatabase.mdb for AC2-B
Processing 13/78: AC3-A
Could not find SecurityDatabase.mdb for AC3-A
Processing 14/78: AC3-B
Could not find SecurityDatabase.mdb for AC3-B
Processing 15/78: AC4-A
Could not find SecurityDatabase.mdb for AC4-A


INFO = Successfully copies SecurityDatabase.mdb for AC4-A to \\ut02sa7\ac\~Cody\Testing\FCC\AC4-A
INFO = Successfully copies SecurityDatabase.mdb for AC4-B to \\ut02sa7\ac\~Cody\Testing\FCC\AC4-B
INFO = Successfully copies SecurityDatabase.mdb for AC6-A to \\ut02sa7\ac\~Cody\Testing\FCC\AC6-A
INFO = Successfully copies SecurityDatabase.mdb for AC6-B to \\ut02sa7\ac\~Cody\Testing\FCC\AC6-B


...

Processing 48/78: COV2-B
Could not find SecurityDatabase.mdb for COV2-B
Processing 49/78: HEOV BB-A
Could not find SecurityDatabase.mdb for HEOV BB-A
Processing 50/78: HEOV BB-B
Could not find SecurityDatabase.mdb for HEOV BB-B
Processing 51/78: HEOV-A
Could not find SecurityDatabase.mdb for HEOV-A
Processing 52/78: HEOV-B
Could not find SecurityDatabase.mdb for HEOV-B
Processing 53/78: HTS Oven-A
Could not find SecurityDatabase.mdb for HTS Oven-A
Processing 54/78: HTS Oven-B
Could not find SecurityDatabase.mdb for HTS Oven-B
Processing 55/78: MBC1-A
Could not find SecurityDatabase.mdb for MBC1-A
Processing 56/78: MBC1-B
Could not find SecurityDatabase.mdb for MBC1-B
Processing 57/78: OV22-A
Could not find SecurityDatabase.mdb for OV22-A
Processing 58/78: OV22-B
Could not find SecurityDatabase.mdb for OV22-B
Processing 59/78: OV25-A
Could not find SecurityDatabase.mdb for OV25-A
Processing 60/78: OV30-A
Could not find SecurityDatabase.mdb for OV30-A
Processing 61/78: OV30-B
Could not f

INFO = Successfully copies SecurityDatabase.mdb for OV51-A to \\ut02sa7\ac\~Cody\Testing\EastGate\OV51-A
INFO = Successfully copies SecurityDatabase.mdb for OV51-B to \\ut02sa7\ac\~Cody\Testing\EastGate\OV51-B


Processing 70/78: OV51-B
Could not find SecurityDatabase.mdb for OV51-B
Processing 71/78: OV52-A


INFO = Successfully copies SecurityDatabase.mdb for OV52-A to \\ut02sa7\ac\~Cody\Testing\Oven22\OV52-A
INFO = Successfully copies SecurityDatabase.mdb for OV52-B to \\ut02sa7\ac\~Cody\Testing\Oven22\OV52-B
INFO = Successfully copies SecurityDatabase.mdb for OV59-A to \\ut02sa7\ac\~Cody\Testing\ACS\OV59-A
INFO = Successfully copies SecurityDatabase.mdb for OV59-B to \\ut02sa7\ac\~Cody\Testing\ACS\OV59-B
INFO = Successfully copies SecurityDatabase.mdb for Spike Oven-A to \\ut02sa7\ac\~Cody\Testing\ACCE\Spike Oven-A
INFO = Successfully copies SecurityDatabase.mdb for Spike Oven-B to \\ut02sa7\ac\~Cody\Testing\ACCE\Spike Oven-B
INFO = ============================================================
INFO =  Finished copying SecurityDatabase.mdb to local machines. Success: 63, Failure: 0
INFO = ============================================================


Could not find SecurityDatabase.mdb for OV52-A
Processing 72/78: OV52-B
Could not find SecurityDatabase.mdb for OV52-B
Processing 73/78: OV59-A
Could not find SecurityDatabase.mdb for OV59-A
Processing 74/78: OV59-B
Could not find SecurityDatabase.mdb for OV59-B
Processing 75/78: Old Lewco-A
Could not find SecurityDatabase.mdb for Old Lewco-A
Processing 76/78: Old Lewco-B
Could not find SecurityDatabase.mdb for Old Lewco-B
Processing 77/78: Spike Oven-A
Could not find SecurityDatabase.mdb for Spike Oven-A
Processing 78/78: Spike Oven-B
Could not find SecurityDatabase.mdb for Spike Oven-B


## Explore Data

## Output